In [1]:
import numpy as np
import pandas as pd
import pystan
import pickle
import gzip

In [2]:
stan_code = """
parameters {
    real a;
    real b[3];
    vector[3] c;
    vector[3] c1[3];
    matrix[3,3] d;
    matrix[4,3] e;
    matrix[3,3] f[5];
    matrix[4,3] g[6, 4];
}
model {
    a ~ normal(0,1);
    b ~ normal(0,1);
    c ~ normal(0,1);
    for (n in 1:3) {
        c1[n] ~ normal(0,1);
        for (m in 1:3) {
            d[n,m] ~ normal(0,1);
        }
    }
    for (n in 1:4) {
        for (m in 1:3) {
            e[n,m] ~ normal(0,1);
        }
    }
    for (k in 1:5) {
        for (n in 1:3) {
            for (m in 1:3) {
                f[k,n,m] ~ normal(0,1);
            }
        }
    }
    for (k in 1:6) {
        for (l in 1:4) {
            for (n in 1:4) {
                for (m in 1:3) {
                    g[k,l,n,m] ~ normal(0,1);
                }
            }
        }
    }
}
generated quantities {
    int h;
    int i[3];
    int j[3,4];
    h = poisson_rng(3);
    for (n in 1:3) {
        i[n] = poisson_rng(3);
        for (m in 1:4) {
            j[n,m] = poisson_rng(3);
        }
    }
}
"""

In [3]:
%time stan_model = pystan.StanModel(model_code=stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_d74ce66b2c5e650c837eafb7417e07c5 NOW.


Wall time: 50.4 s


In [4]:
%%time
%time fit0 = stan_model.sampling(seed=131, check_hmc_diagnostics=False)
%time fit1 = stan_model.sampling(chains=1, seed=132, check_hmc_diagnostics=False)
%time fit2 = stan_model.sampling(iter=10, seed=133, check_hmc_diagnostics=False)
%time fit3 = stan_model.sampling(iter=10, chains=1, seed=134, check_hmc_diagnostics=False)
%time fit4 = stan_model.sampling(iter=1, warmup=0, seed=135, check_hmc_diagnostics=False)
%time fit5 = stan_model.sampling(iter=1, warmup=0, chains=1, seed=136, check_hmc_diagnostics=False)
%time fit6 = stan_model.sampling(iter=1, chains=1, algorithm="Fixed_param", seed=137, check_hmc_diagnostics=False)
%time fit7 = stan_model.sampling(iter=100, chains=1, algorithm="Fixed_param", seed=138, check_hmc_diagnostics=False)

Wall time: 7.65 s
Wall time: 2.38 s
Wall time: 3.12 s
Wall time: 20 ms


Wall time: 3.48 s
Wall time: 13 ms
Wall time: 7 ms
Wall time: 16 ms
Wall time: 16.7 s


In [5]:
%%time
import sys
with gzip.open(f"./model_and_fits_{pystan.__version__}_{sys.platform}.pickle.gzip", "wb") as f:
    f.write(pickle.dumps([stan_model, fit0, fit1, fit2, fit3, fit4, fit5, fit6, fit7]))

C:\Users\aohartik\miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  This is separate from the ipykernel package so we can avoid doing imports until


Wall time: 2.54 s


In [6]:
%%time
extracts = {}
for fit, key in zip([fit0, fit1, fit2, fit3, fit4, fit5, fit6, fit7], 
                    ['basic', 'basic_chain1', 'iter10', 'iter10chain1', 
                     'iter1_chain4', 'iter1chain1', 'fixed_paramchain_n1', 'fixed_param_n100']):
    pars = fit.model_pars + ['lp__']
    extracts[key] = {'permuted' : fit.extract(),
                     'not_permuted' : fit.extract(pars=pars, permuted=False), 
                     'ndarray' : fit.extract(permuted=False),
                    }

Wall time: 1.02 s


In [7]:
%%time
with gzip.open(f"./shape_comparison_test_{pystan.__version__}.pickle.gzip", "wb") as f:
    f.write(pickle.dumps(extracts))

Wall time: 2.97 s
